<a href="https://colab.research.google.com/github/leticiasdd/Proj_aplicado_III/blob/main/Projeto_aplicado_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**RECOMENDAÇÃO DE LIVROS COM BASE NA PREFERÊNCIA CINEMATOGRÁFICA DO USUÁRIO**


Bruno Luigi dos Santos Tobias | RA 10423789

Letícia Serrano | RA 10415844

Lucas Vaz | RA 10424623

Lorena Vaz Cord | RA 10424700

##Datasets

###Livros
- https://cseweb.ucsd.edu/~jmcauley/datasets/goodreads.html
- Baixar o arquivo: Detailed book graph (~2gb, about 2.3m books): goodreads_books.json.gz

###Filmes
- https://www.kaggle.com/datasets/gsimonx37/letterboxd?select=movies.csv
- Baixar o arquivo: movies.csv

###Instruções para execução do código:
- Os datasets originais devem estar armazenados no Google Drive.
- O caminho para os arquivos deve ser: Projeto aplicado 3/datasets/

## Conexão com o Google Drive

Esse metodo não funciona com arquivos compartilhados, é preciso ter uma cópia dos arquivos na pasta correta do Google Drive.

In [ ]:
# Montar o drive do Google
# Após rodar esse código, ele pedirá autorização para acessar o Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Definir caminho para o dataset
DIR = '/content/drive/MyDrive/Projeto aplicado 3/datasets/'

##Importação das bibliotecas

In [ ]:
# Importar bibliotecas
import gzip
import json
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

##Importação e pré processamento do dataset de livros

###Análise inicial do conteúdo do dataset de livros

In [ ]:
# Carregar as primeiras 10 linhas do JSON compactado
def load_first_n(file_name, n=10):
    data = []
    with gzip.open(file_name) as fin:
        for i, l in enumerate(fin):
            if i >= n:
                break
            data.append(json.loads(l))
    return pd.DataFrame(data)

# Contar o numero de registros no JSON compactado
def count_lines_in_gzip(file_path):
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        return sum(1 for _ in f)

# Carrega as primeiras 10 linhas do arquivo
file_path = '/content/drive/MyDrive/Projeto aplicado 3/datasets/goodreads_books.json.gz'
df_sample = load_first_n(file_path, 10)

In [ ]:
# Quantidade de registros e colunas
# Esta etapa pode demorar alguns minutos para executar devido a quantidade de registros
print(f"\nNúmero de registros: {count_lines_in_gzip(file_path)}")
print(f"Número de colunas: {len(df_sample.columns)}")


Número de registros: 2360655
Número de colunas: 29


In [ ]:
# Lista de colunas no JSON compactado
print("\nColunas presentes no dataset de livros:")
for col in df_sample.columns:
    print(f" - {col}")


Colunas presentes no dataset de livros:
 - isbn
 - text_reviews_count
 - series
 - country_code
 - language_code
 - popular_shelves
 - asin
 - is_ebook
 - average_rating
 - kindle_asin
 - similar_books
 - description
 - format
 - link
 - authors
 - publisher
 - num_pages
 - publication_day
 - isbn13
 - publication_month
 - edition_information
 - publication_year
 - url
 - image_url
 - book_id
 - ratings_count
 - work_id
 - title
 - title_without_series


In [ ]:
# Mostrar as primeiras linhas
print("Exemplo de registros:")
print(df_sample.head())

Exemplo de registros:
         isbn text_reviews_count    series country_code language_code  \
0  0312853122                  1        []           US                 
1  0743509986                  6        []           US                 
2                              7  [189911]           US           eng   
3  0743294297               3282        []           US           eng   
4  0850308712                  5        []           US                 

                                     popular_shelves        asin is_ebook  \
0  [{'count': '3', 'name': 'to-read'}, {'count': ...                false   
1  [{'count': '2634', 'name': 'to-read'}, {'count...                false   
2  [{'count': '58', 'name': 'to-read'}, {'count':...  B00071IKUY    false   
3  [{'count': '7615', 'name': 'to-read'}, {'count...                false   
4  [{'count': '32', 'name': 'to-read'}, {'count':...                false   

  average_rating kindle_asin  ... publication_month edition_information  \
0

###Definição das funções que serão utilizadas na preparação do dataset de livros

Devido ao grande volume do dataset original de livros, optamos por realizar o pré-processamento em blocos (chunks). Isso evita problemas de memória durante a manipulação dos dados, permitindo processar o arquivo em partes gerenciáveis.

In [ ]:
# Função para carregamento do arquivo em chunks (adaptada de https://cseweb.ucsd.edu/~jmcauley/datasets/goodreads.html)
def load_data(file_name, chunk_size = 500):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)

            if len(data) >= chunk_size:
                yield data
                data = []  # Limpa o buffer para o próximo chunk

        if data:       # Retorna os dados restantes, se houver
            yield data

As funções abaixo serão utilizadas para limpar e padronizar dados.

In [ ]:
# Função para tratar o campo do número de páginas do livro
# Se a string estiver vazia (''), retona NaN, se não, converte a string em número inteiro
def fix_pages(pages):
    if pages == '':
        return np.nan
    else:
        return int(pages)

# Função para tratar o campo de nota de avaliação do livro
# Se não conseguir converter o valor em número decimal, converte o valor para NaN
def filter_rating(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

# Definição de parametros para filtrar o DataFrame de livros para deixar apenas os registros que são úteis para análise
languages_list = ['eng','en-US','en-GB','en-CA','en']
undesired_format = ['Audio CD','Audio','Audiobook','Audible Audio','MP3 CD','Audio Cassette','Overdrive Audiobook','online fanfiction','online fiction','online short story']
rating_cutoff = 3.5
min_lim_pages = 50
desired_attributtes = ['description','publication_year','url','work_id','title']


# Funções para limpar e filtrar o DataFrame de livros para deixar apenas os registros que são úteis para análise
def treat_data(df, languages_list, undesired_format, rating_cutoff, min_lim_pages, desired_attributtes):
    # Filtro de idioma
    df = df[df['language_code'].isin(languages_list)]

    # Filtro de formato da midia
    df = df[-df['format'].isin(undesired_format)]

    # Filtro de nota média de avaliação
    df.loc[:, 'average_rating'] = df['average_rating'].apply(filter_rating)
    df = df[df['average_rating'] >= rating_cutoff]

    # Filtro de número mínimo de páginas
    df.loc[:, 'num_pages'] = df['num_pages'].apply(fix_pages)
    df = df[(df['num_pages'].isna()) | (df['num_pages'] > min_lim_pages)]

    # Filtro para remover livros que não contém descrição
    df = df[df['description'] != '']

    # Filtro para remoção de livros com títulos duplicados
    # Se houver dois ou mais livros com o mesmo título, ele remove os duplicados, mantendo o primeiro que aparecer no DataFrame
    df = df.drop_duplicates(subset=['title'])

    # Seleção de colunas utilizadas na análise
    df = df[desired_attributtes]

    return df

Reduzimos o número de colunas para incluir apenas os atributos essenciais para a análise posterior, como título e descrição do livro. Essa redução simplifica o dataset, diminui o uso de memória.

O tratamento também inclui filtros que removem registros com idiomas não relevantes, formatos de mídia indesejados, livros com nota de avaliação baixa, livros com pouco número de páginas e livros sem descrição.


##Processamento do dataset em blocos (chunks)

O arquivo JSON é lido em blocos de 100.000 registros por vez. A cada bloco lido, são aplicados os filtros e as transformações definidos nas funções anteriores.

Após o processamento, cada bloco é salvo separadamente como um arquivo no formato .csv.

In [ ]:
# Esta etapa pode demorar alguns minutos para executar devido a quantidade de registros
# Pasta onde os blocos csv serão salvos
output_dir = '/content/drive/MyDrive/Projeto aplicado 3/datasets/dataset_tratado_books/'

# Cria a pasta se não existir
os.makedirs(output_dir, exist_ok=True)

# Tratamento e exportação do dataset de livros em chunks
ct = 0
total_records = 0
for chunk in load_data(os.path.join(DIR, 'goodreads_books.json.gz'), chunk_size=100000):
    df_book_raw = pd.DataFrame(chunk)
    df_book_raw = treat_data(df_book_raw, languages_list, undesired_format, rating_cutoff, min_lim_pages, desired_attributtes)
    ct += 1
    total_records += len(df_book_raw)
    output_path = os.path.join(output_dir, f'chunk_{ct}.csv')
    df_book_raw.to_csv(output_path, index=False)
    print(f"Chunk {ct} salvo com {len(df_book_raw)} registros.")

print(f"\nProcessamento finalizado. Total de registros processados: {total_records}")

Chunk 1 salvo com 25666 registros.
Chunk 2 salvo com 25840 registros.
Chunk 3 salvo com 25798 registros.
Chunk 4 salvo com 25225 registros.
Chunk 5 salvo com 25292 registros.
Chunk 6 salvo com 25460 registros.
Chunk 7 salvo com 25216 registros.
Chunk 8 salvo com 25493 registros.
Chunk 9 salvo com 25298 registros.
Chunk 10 salvo com 25351 registros.
Chunk 11 salvo com 25509 registros.
Chunk 12 salvo com 25516 registros.
Chunk 13 salvo com 25868 registros.
Chunk 14 salvo com 25921 registros.
Chunk 15 salvo com 25623 registros.
Chunk 16 salvo com 25664 registros.
Chunk 17 salvo com 25386 registros.
Chunk 18 salvo com 25521 registros.
Chunk 19 salvo com 25322 registros.
Chunk 20 salvo com 25475 registros.
Chunk 21 salvo com 25257 registros.
Chunk 22 salvo com 25408 registros.
Chunk 23 salvo com 25448 registros.
Chunk 24 salvo com 15352 registros.

Processamento finalizado. Total de registros processados: 601909


Optamos por salvar os arquivos em blocos pré-processados no formato CSV como uma etapa intermediária no fluxo de tratamento dos dados. Essa abordagem permite que o pré-processamento seja executado apenas uma vez, evitando a repetição de etapas onerosas.

Além disso, caso ocorra alguma falha, interrupção ou necessidade de ajustes posteriores, é possível retomar o processo a partir dos arquivos já salvos, sem a necessidade de reprocessar o dataset bruto original.

##Importação e pré processamento do dataset de filmes

###Carregamento e análise inicial do conteúdo do dataset de filmes

In [ ]:
# Caminho do arquivo
movies_zip_path = os.path.join(DIR, 'movies.csv.zip')

# Carregar o CSV diretamente do arquivo ZIP
df_film_raw = pd.read_csv(movies_zip_path)
print(f"\nProcessamento finalizado. Total de registros processados: {len(df_film_raw)}")


Processamento finalizado. Total de registros processados: 941597


In [ ]:
# Quantidade de registros e colunas
print(f"Número de registros: {len(df_film_raw)}")
print(f"Número de colunas: {len(df_film_raw.columns)}")

Número de registros: 941597
Número de colunas: 7


In [ ]:
# Lista de colunas
print("Colunas presentes no dataset de filmes:")
for col in df_film_raw.columns:
    print(f" - {col}")

Colunas presentes no dataset de filmes:
 - id
 - name
 - date
 - tagline
 - description
 - minute
 - rating


In [ ]:
# Visualizar as primeiras linhas
print(df_film_raw.head())

        id                               name    date  \
0  1000001                             Barbie  2023.0   
1  1000002                           Parasite  2019.0   
2  1000003  Everything Everywhere All at Once  2022.0   
3  1000004                         Fight Club  1999.0   
4  1000005                         La La Land  2016.0   

                                            tagline  \
0                  She's everything. He's just Ken.   
1                       Act like you own the place.   
2  The universe is so much bigger than you realize.   
3                           Mischief. Mayhem. Soap.   
4                    Here's to the fools who dream.   

                                         description  minute  rating  
0  Barbie and Ken are having the time of their li...   114.0    3.86  
1  All unemployed, Ki-taek's family takes peculia...   133.0    4.56  
2  An aging Chinese immigrant is swept up in an i...   140.0    4.30  
3  A ticking-time-bomb insomniac and a sli

In [ ]:
# Verificar valores duplicados na coluna id
sum(df_film_raw['id'].duplicated())

0

In [ ]:
# Verificar valores ausentes em cada coluna
print("Valores ausentes por coluna:")
print(df_film_raw.isna().sum())

# Verificar o total de valores ausentes no DataFrame
total_missing = df_film_raw.isna().sum().sum()
print(f"\nTotal de valores ausentes no dataset: {total_missing}")

Valores ausentes por coluna:
id                  0
name               10
date            91913
tagline        802210
description    160812
minute         181570
rating         850598
dtype: int64

Total de valores ausentes no dataset: 2087113


###Preparação do dataset de filmes

In [ ]:
# Remover registros de filmes sem nome
df_film_raw.dropna(subset=['name'], inplace=True)

# Remover registros de filmes sem descrição
df_film_raw.dropna(subset=['description'], inplace=True)

# Exibir quantidade de registros final
print(f"Total de registros no dataset: {len(df_film_raw)}")

Total de registros no dataset: 780783


Durante a análise inicial do dataset de filmes, observamos a presença de registros com dados ausentes, como a identificação única de um filme depende tanto de seu nome quanto da sua data, optamos por priorizar os registros que contêm ambas as informações.

Dessa forma, realizamos a ordenação do dataset de forma que, para cada conjunto de filmes com o mesmo nome, os registros com data de lançamento aparecessem primeiro. Em seguida, removemos os registros duplicados com base na coluna name, mantendo o primeiro registro de cada grupo.

In [ ]:
# Remover filmes com nomes duplicados
# Ordenar o DataFrame para garantir que os registros com data apareçam antes dos que não têm data
df_film_raw = df_film_raw.sort_values(by=['name', 'date'], ascending=[True, False])

# Criar uma coluna auxiliar para indicar se há data (True/False)
df_film_raw['has_date'] = df_film_raw['date'].notna()

# Remover registros duplicados mantendo o primeiro registro por nome, priorizando os que têm data
df_film_raw = df_film_raw.drop_duplicates(subset=['name'], keep='first')

# Remover a coluna auxiliar
df_film_raw = df_film_raw.drop(columns=['has_date'])

# Exibir quantidade de registros final
print(f"Total de registros no dataset após remover duplicados: {len(df_film_raw)}")

Total de registros no dataset após remover duplicados: 652839


In [ ]:
# Seleção de colunas utilizadas na análise
df_film_raw = df_film_raw[['id', 'name', 'date', 'tagline', 'description']]

Reduzimos o número de colunas para incluir apenas os atributos essenciais para a análise posterior, como nome do filme e descrição.

##Processamento do dataset de filmes

Após a realização das etapas de limpeza e tratamento dos dados do dataset de filmes — incluindo a remoção de registros com campos essenciais ausentes, como nome e descrição, e a eliminação de duplicatas priorizando os registros mais completos — optamos por salvar o dataset tratado em formato CSV.

Essa decisão segue o mesmo padrão adotado durante o processamento do dataset de livros e tem como principal objetivos padronizar o fluxo de trabalho.

In [ ]:
# Pasta onde o csv tratato será salvo
output_dir_film = '/content/drive/MyDrive/Projeto aplicado 3/datasets/dataset_tratado_films'

# Criar a pasta se não existir
os.makedirs(output_dir_film, exist_ok=True)

# Definição do nome do csv
output_path_film = os.path.join(output_dir_film, 'films_treated.csv')

# Exportar o DataFrame como CSV
df_film_raw.to_csv(output_path_film, index=False)

print(f"\nProcessamento finalizado. Total de registros processados: {len(df_film_raw)}")


Processamento finalizado. Total de registros processados: 652839


##Carregamento dos dados tratatos

###Unificação e carregamento dos blocos com os dados tratados de livros

In [ ]:
# Caminho para os chunks de livros no Google Drive
books_dir = '/content/drive/MyDrive/Projeto aplicado 3/datasets/dataset_tratado_books/'

# Lista os arquivos da pasta
book_files = os.listdir(books_dir)

# DataFrame vazio para acumular os dados
df_book = pd.DataFrame()

# Loop para carregar e concatenar os chunks
for file in book_files:
    if 'chunk' in file and file.endswith('.csv'):
        file_path = os.path.join(books_dir, file)
        df_chunk = pd.read_csv(file_path)
        df_book = pd.concat([df_book, df_chunk], ignore_index=True)
        print(f"Carregado: {file} ({len(df_chunk)} registros)")

print(f"\nTotal de registros de livros: {len(df_book)}")

Carregado: chunk_1.csv (25666 registros)
Carregado: chunk_2.csv (25840 registros)
Carregado: chunk_3.csv (25798 registros)
Carregado: chunk_4.csv (25225 registros)
Carregado: chunk_5.csv (25292 registros)
Carregado: chunk_6.csv (25460 registros)
Carregado: chunk_7.csv (25216 registros)
Carregado: chunk_8.csv (25493 registros)
Carregado: chunk_9.csv (25298 registros)
Carregado: chunk_10.csv (25351 registros)
Carregado: chunk_11.csv (25509 registros)
Carregado: chunk_12.csv (25516 registros)
Carregado: chunk_13.csv (25868 registros)
Carregado: chunk_14.csv (25921 registros)
Carregado: chunk_15.csv (25623 registros)
Carregado: chunk_16.csv (25664 registros)
Carregado: chunk_17.csv (25386 registros)
Carregado: chunk_18.csv (25521 registros)
Carregado: chunk_19.csv (25322 registros)
Carregado: chunk_20.csv (25475 registros)
Carregado: chunk_21.csv (25257 registros)
Carregado: chunk_22.csv (25408 registros)
Carregado: chunk_23.csv (25448 registros)
Carregado: chunk_24.csv (15352 registros)



In [ ]:
# Visualizar as primeiras linhas
print(df_book.head())

                                         description  publication_year  \
0  Omnibus book club edition containing the Ladie...            1987.0   
1  What is Heaven really going to be like? What w...               NaN   
2  A stunning and revealing examination of oil's ...               NaN   
3  Number 30 in a series of literary pamphlets pu...            1887.0   
4  Embrace the word of God with the inspirational...            2013.0   

                                                 url   work_id  \
0  https://www.goodreads.com/book/show/7327624-th...   8948723   
1   https://www.goodreads.com/book/show/89376.Heaven     86257   
2  https://www.goodreads.com/book/show/6158967-cr...   6338156   
3  https://www.goodreads.com/book/show/16037549-v...   5212748   
4  https://www.goodreads.com/book/show/18628482-u...  26419472   

                                               title  
0  The Unschooled Wizard (Sun Wolf and Starhawk, ...  
1                                             He

###Carregamento dos dados tratados de filmes

In [ ]:
# Caminho do arquivo de filmes Google Drive
file_path = '/content/drive/MyDrive/Projeto aplicado 3/datasets/dataset_tratado_films/films_treated.csv'

# Carregar o CSV
df_film = pd.read_csv(file_path)

# Exibir quantidade de registros carregados
print(f"Total de registros carregados: {len(df_film)}")

Total de registros carregados: 652839


In [ ]:
# Visualizar as primeiras linhas
print(df_film.head())

        id                                   name    date  \
0  1516866                                      !  2023.0   
1  1826456                   ! (Exclamation Mark)  2023.0   
2  1771719  ! BONUS VIDEO ! - A Melee Combo Video  2023.0   
3  1320487                          ! EXCLAMATION  1976.0   
4  1392532                              ! [ai-ou]  1991.0   

                    tagline  \
0                       NaN   
1                       NaN   
2                       NaN   
3             ! EXCLAMATION   
4  気づいてほしい･･･ [ai-ou] 心の叫び。   

                                         description  minute  rating  \
0  Follows the rising panic of having something t...     3.0     NaN   
1  An original short film that explores important...     4.0     NaN   
2  Sequel to Annihilation Domination (Artistic en...     2.0     NaN   
3  This short film tells a story in the form of a...    17.0     NaN   
4  Three outsiders meet by chance and try to chas...   108.0     NaN   

             

##Execução do modelo

Para que o modelo de recomendação baseado em similaridade textual funcione de forma eficiente, é essencial que o vocabulário utilizado pelo vetorizar TF-IDF seja construído a partir de todo o conteúdo disponível. Isso garante que os termos presentes tanto nos livros quanto nos filmes sejam considerados de maneira unificada, possibilitando uma comparação justa entre os dois conjuntos.

Nesta etapa, realizamos a concatenação dos principais atributos textuais dos livros e filmes em uma nova coluna chamada tags, unificando título, descrição e, no caso dos filmes, também o tagline. Em seguida, essas colunas são combinadas em uma única série (all_tags), que será usada para construir o vocabulário do TF-IDF.

Dessa forma, o modelo consegue mapear relações entre obras com base em descrições semelhantes, mesmo que estejam em domínios diferentes (literatura e cinema).

In [ ]:
# Concatenando os atributos do dataset de livros
df_book['tags'] = df_book['title'].fillna("") + ' ' + df_book['description'].fillna("")

# Concatenando os atributos do dataset de filmes
df_film['tags'] = df_film['name'].fillna("") + ' ' + df_film['tagline'].fillna("") + ' ' + df_film['description'].fillna("")

# Concatenar as colunas de tags de ambos os datasets
all_tags = pd.concat([df_film['tags'], df_book['tags']], ignore_index=True)

Com a série all_tags unificando as descrições de filmes e livros, iniciamos o processo de vetorização utilizando o modelo TF-IDF, uma técnica amplamente usada para representar textos numericamente com base na relevância de cada termo em relação ao corpus total.

Primeiramente, instanciamos o TfidfVectorizer, que é responsável por transformar os textos em vetores numéricos. Utilizamos o parâmetro stop_words='english' para remover palavras comuns da língua inglesa que não agregam significado (como "the", "and", "of", etc.), reduzindo o ruído nos dados.

Em seguida, ajustamos o vetorizador (fit) usando todo o conteúdo combinado de filmes e livros, garantindo que o vocabulário aprendido seja global e compartilhe a mesma base de termos. Isso é essencial para possibilitar a comparação entre obras de diferentes domínios no mesmo espaço vetorial.

Após o ajuste, aplicamos a transformação (transform) separadamente nos conjuntos de filmes e livros. O resultado são dois conjuntos de vetores TF-IDF (film_tfidf e book_tfidf), que representam, de forma numérica, o conteúdo textual de cada item.

In [ ]:
# Essa etapa pode demorar alguns minutos
# Criar e ajustar o vetorizador com todas as tags (de filmes e livros)
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit(all_tags)

# Transformar cada dataset separadamente com o mesmo vocabulário treinado
film_tfidf = vectorizer.transform(df_film['tags'])
book_tfidf = vectorizer.transform(df_book['tags'])

Por último, a função recomendation recebe o nome de um filme e retorna uma lista dos 10 livros mais similares a ele, com base na similaridade dos conteúdos textuais (usando TF-IDF e similaridade cosseno).

In [ ]:
def recomendation(film_title):
    matches = df_film[df_film['name'].str.lower() == film_title.lower()]

    if matches.empty:
        return f"Filme '{film_title}' não encontrado no dataset."

    i_film = matches.index[0]
    similaridades = cosine_similarity(film_tfidf[i_film], book_tfidf)
    i_books = similaridades.argsort()[0][-10:][::-1]
    return df_book.iloc[i_books][['title', 'url']]

Agora podemos inserir o nome de um filme e recerber recomendações de livros similares.

In [ ]:
print(recomendation('Conclave'))

                                                    title  \
255494  The Francis Miracle: Inside the Transformation...   
251242                                      Peter’s Chair   
412875                        The Making of the Pope 2005   
540707  Strange Gods:  A Novel About Faith, Murder, Si...   
140645  Pope to the Poor: The Life and Times of Pope F...   
13955           What Has Happened to the Catholic Church?   
211669           The Secret Cardinal (Nolan Kilkenny, #5)   
159095                       Francis: Pope of a New World   
167               The Possession of Lawrence Eugene Davis   
516721  Render Unto Rome: The Secret Life of Money in ...   

                                                      url  
255494  https://www.goodreads.com/book/show/20898083-t...  
251242  https://www.goodreads.com/book/show/7081044-pe...  
412875  https://www.goodreads.com/book/show/1274463.Th...  
540707  https://www.goodreads.com/book/show/28175644-s...  
140645  https://www.goodread